In [1]:
# %load train.py
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, LSTM
from keras.models import Model
from sklearn import metrics
import time
import sys

#########################################################################

In [2]:
data_name = 'feature.csv'
drop_column = ['Unnamed: 0']
reshape_value = 34
'''
#  0 ~ 27 : time slot 0 ~ 27
# 28 : continuous week
# 29 ~ 31 : Platform
# 32 ~ 33 : connect type
# 34 : watch ratio
# 35 : hot drama
# 36 : new episode next week
'''
shape_num = 38 - len(drop_column)

In [3]:
data = pd.read_csv(data_name)
print(data.shape)
data = data.drop(columns = drop_column)
print(data.shape)

(1943406, 38)
(1943406, 37)


In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data = data.to_numpy()
data = data.reshape(-1, reshape_value, shape_num)

In [6]:
data.shape  #total user: 57159

(57159, 34, 37)

In [7]:

n_data = []
for line in data :
    line = np.delete(line, -1, axis=0)
    line = np.delete(line, -1, axis=0)
    n_data.append(line)

n_data = np.array(n_data)



In [8]:
n_data.shape

(57159, 32, 37)

In [10]:
label = pd.read_csv('labels.csv')
label = label.drop(columns=['1', '2'])
label = label.to_numpy()

In [11]:

x_, x_test , y_, y_test = train_test_split(n_data, label, test_size=0.3, shuffle=True, random_state=110)
x_train, x_val, y_train, y_val = train_test_split(x_, y_, test_size=0.3, shuffle=True, random_state=30)
'''
x_train = n_data[:35001, :, :]
x_test = n_data[35001: , :, :]
y_train = label[:35001, :]
y_test = label[35001: , :]
'''

'\nx_train = n_data[:35001, :, :]\nx_test = n_data[35001: , :, :]\ny_train = label[:35001, :]\ny_test = label[35001: , :]\n'

In [12]:
x = sequence_input = Input(shape=(32, shape_num))

x = LSTM(128, dropout=0.375)(x)
x = Dense(128, activation='relu')(x)
preds = Dense(28, activation='sigmoid')(x)

model = Model(sequence_input, preds)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])







In [13]:
start_time = time.asctime( time.localtime(time.time()) )
model.fit(x_train, y_train, shuffle=True, epochs=100, batch_size=32, validation_data = (x_val, y_val))

Epoch 1/100
876/876 [==============================] - 52s 59ms/step - loss: 0.2383 - acc: 0.0482 - val_loss: 0.2179 - val_acc: 0.0405
Epoch 2/100
876/876 [==============================] - 48s 55ms/step - loss: 0.2193 - acc: 0.0527 - val_loss: 0.2165 - val_acc: 0.0361
Epoch 3/100
876/876 [==============================] - 47s 54ms/step - loss: 0.2175 - acc: 0.0574 - val_loss: 0.2129 - val_acc: 0.0624
Epoch 4/100
876/876 [==============================] - 47s 53ms/step - loss: 0.2158 - acc: 0.0651 - val_loss: 0.2120 - val_acc: 0.0867
Epoch 5/100
876/876 [==============================] - 43s 49ms/step - loss: 0.2146 - acc: 0.0708 - val_loss: 0.2102 - val_acc: 0.0420
Epoch 6/100
876/876 [==============================] - 44s 50ms/step - loss: 0.2139 - acc: 0.0671 - val_loss: 0.2102 - val_acc: 0.0950
Epoch 7/100
876/876 [==============================] - 45s 52ms/step - loss: 0.2129 - acc: 0.0692 - val_loss: 0.2085 - val_acc: 0.0633
Epoch 8/100
876/876 [==============================] - 

In [14]:
model.save('kktv.h5')

In [21]:
from keras.models import load_model
model=load_model('kktv.h5')

In [22]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 37)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               84992     
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 28)                3612      
Total params: 105,116
Trainable params: 105,116
Non-trainable params: 0
_________________________________________________________________


In [15]:
score = model.evaluate(n_data, label)

print('Test loss : ', score[0])
print('Test accuracy : ', score[1])

1787/1787 [==============================] - 29s 16ms/step - loss: 0.1976 - acc: 0.1049
Test loss :  0.19762766361236572
Test accuracy :  0.10493535548448563


In [16]:
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)

'''
train_predict = train_predict.reshape(-1, 1)
test_predict = test_predict.reshape(-1, 1)
y_predict = y_predict.reshape(-1,1)
label = label.reshape(-1,1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
'''
print('Train AUC : ', metrics.roc_auc_score(y_train, train_predict))
print('Test AUC : ', metrics.roc_auc_score(y_test, test_predict))



print('Execute from ', start_time, 'to', time.asctime( time.localtime(time.time()) ))

Train AUC :  0.9317892317930275
Test AUC :  0.8311334881848944
Execute from  Thu Oct 29 15:23:38 2020 to Thu Oct 29 16:38:53 2020


In [17]:
predict_y = model.predict(x_train[8:9])
predict_y


array([[0.03518334, 0.8494363 , 0.00840411, 0.04767689, 0.02338672,
        0.8758188 , 0.00233707, 0.08195284, 0.0264883 , 0.79590094,
        0.00411975, 0.02428538, 0.03484058, 0.78319466, 0.00259039,
        0.06341517, 0.0484232 , 0.6941031 , 0.02720025, 0.25332913,
        0.6262193 , 0.8477131 , 0.22668076, 0.30748177, 0.6669688 ,
        0.820618  , 0.00948685, 0.00962505]], dtype=float32)

In [18]:
y_train[8:9]

array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 0]], dtype=int64)

In [19]:
pre=np.where(predict_y>1,1,0)
pre

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [20]:
np.where(pre!=y_train[8:9])[1]

array([ 1,  5,  9, 13, 19, 20, 21, 23, 24, 25], dtype=int64)